In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel



# Read the CSV file
df = pd.read_csv("cement.csv")
applications = df['Application'].unique()
print(applications)

search_materials = ['Foundation', 'Walls', 'Roofing', 'Flooring', 'Plastering', 'Repairs', 'Mass Concrete']

def filter_material(application, material):
    c_df = df[df['Application'] == application]
    neces_df = c_df[['Material', 'Review']]  # Select Material and Review
    tfv = TfidfVectorizer(min_df=2, max_features=None,
                        strip_accents='unicode', analyzer='word',
                        token_pattern=r'\w{1,}', ngram_range=(1, 3),
                        stop_words='english')
    neces_df.loc[:, 'Review'] = neces_df['Review'].fillna('')
    tfv_matrix = tfv.fit_transform(neces_df['Review'])
    sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
    indices = pd.Series(neces_df.index, index=neces_df['Material']).drop_duplicates()
    indices_lower = indices.index.str.lower()

def give_rec(title, sig=sig):
    if title in neces_df['Material'].tolist():
      # Matched Title: Return similar materials for the exact title
        idx = indices[indices_lower.get_loc(title)]
        sig_scores = list(enumerate(sig[idx]))
        sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
        sig_scores = sig_scores[1:-1]  # Exclude the material itself
        n_in = [i[0] for i in sig_scores]
        return df.iloc[n_in]  # Return data from original df using n_in
    else:
      # Unmatched Title: Return application-specific materials sorted by similarity
        all_materials = neces_df.copy()
        all_materials['Similarity Score'] = sig.diagonal()
        all_materials = all_materials.sort_values(by='Similarity Score', ascending=False)
        all_materials.drop('Similarity Score', axis=1, inplace=True)
        return all_materials.merge(df[df['Application'] == application], how='left', on='Material')  # Merge with filtered df for the specified application

  # Call give_rec within filter_material (remove printing)
  all_materials = give_rec(material)

  return all_materials  # Return the DataFrame containing recommendations


# Loop through applications and call filter_material
for material in search_materials:
    material_lower = material.lower()

    for application in applications:
        if df[df['Application'] == application].shape[0] < 3:
            continue
        all_foundations = filter_material(application, material_lower)

        # Print the DataFrame to inspect its structure
        print(f"\n\n** Recommendations for {application.upper()} Application with {material.upper()} material:")
        print(all_foundations)

        # Check if 'Similarity Score' column exists in all_foundations DataFrame
        if 'Similarity Score' in all_foundations.columns:
            plt.figure(figsize=(10, 6))
            plt.bar(all_foundations['Material'], all_foundations['Similarity Score'])
            plt.xlabel('Material')
            plt.ylabel('Similarity Score')
            plt.title(f'Recommendations for {application.upper()} Application with {material.upper()} material')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        else:
            print(f"No 'Similarity Score' column found in recommendations for {application.upper()} Application with {material.upper()} material")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 47)